In [1]:
import visa
import numpy as np
import pandas as pd
import tqdm

In [19]:
#Locale settings
import locale
# Set to German locale to get comma decimal separater
locale.setlocale(locale.LC_NUMERIC, "deu_DEU")

import numpy as np
import matplotlib.pyplot as plt
plt.rcdefaults()

# Tell matplotlib to use the locale we set above
plt.rcParams['axes.formatter.use_locale'] = True

In [20]:
%matplotlib qt
from qsweepy import *
from qsweepy.ponyfiles import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)
from importlib import reload

sample_name='bolometr'

In [21]:
pna = RS_ZNB20('pna', address = 'TCPIP0::10.20.61.161::inst0::INSTR')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [ ]:
pna.set_nop()

In [ ]:
currents = arange(-1.,1, 0.02)*1e-3

pna_preset = True

#cs = Yokogawa_GS210(address='GPIB1::1::INSTR')
#pna = Agilent_N5242A('pna', address = 'ZVB20-23-100170')
if pna_preset:
    pna.set_power(-50.)
    pna.set_bandwidth(400)
    pna.set_centerfreq(6.87656e9)
    pna.set_span(100e6)

coil_device.set_status(1)
sweeper.sweep(pna, (currents, coil_device.set_current, 'Current'))
coil_device.set_status(0)
pna.close()

In [22]:

pna.set_status(1)
pna.set_power(-45) # 
pna.set_nop(1001)
pna.set_bandwidth(100)
pna.set_xlim(4.4345e9, 4.4349e9)
#pna.set_bandwidth(100)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [ ]:
pna.pre_sweep()
plt.figure()
plt.plot(pna.get_points()['S-parameter'][0][1],np.log10(np.abs(pna.measure()['S-parameter']))*10)
pna.post_sweep()pna.pre_sweep()
plt.figure()
plt.plot(pna.get_points()['S-parameter'][0][1],np.log10(np.abs(pna.measure()['S-parameter']))*10)
pna.post_sweep()

In [34]:
S21[0]

array([0.01134432, 0.01107268, 0.01109229, ..., 0.01058339, 0.01097266,
       0.01088017])

In [30]:

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np

df = pd.DataFrame(S21[0],S21[1])
pd.DataFrame()
writer = ExcelWriter('Pandas-Example2.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.save()

In [4]:
vi = visa.ResourceManager()
vna = vi.open_resource('TCPIP0::10.20.61.161::inst0::INSTR')

In [5]:
sample = {'name':'Bolometer-25042019',
          'f_min':1.480653e9,
          'f_max':1.480693e9,
          'bw': 100,
          'nop': 801,
          'power':-40}

In [6]:
powers=np.arange(-55,-30,5)
vna.write('INIT:CONT 0;')
vna.write('SENS:FREQ:STAR {};'.format(sample['f_min']))
vna.write('SENS:FREQ:STOP {};'.format(sample['f_max']))
vna.write('SENS:SWE:POIN {};'.format (sample['nop']))
vna.write('SENS:BAND {};'.format(sample['bw']))
f = np.linspace(sample['f_min'], sample['f_max'], sample['nop'])
sweep_time=float(vna.query('SENS:SWE:TIME?'))
vna.timeout = (sweep_time+1)*1000 # in milliseconds

In [39]:
print(sweep_time)

10.5505


In [15]:
vna.query('*OPC?')

'1\n'

In [15]:
S21_complex = {}

In [16]:
for power in tqdm.tqdm(powers):
    vna.write('SOUR:POW {};'.format(power))
    vna.write('INIT:IMM;')
    result = np.asarray(vna.query_binary_values('CALC:DATA? SDATA;*OPC', datatype=u'f'))
    S21_complex[power] = (result[::2]+1j*result[1::2])
vna.write('INIT:CONT 1;')




  0%|                                                                                            | 0/5 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 73.54it/s]

(14, <StatusCode.success: 0>)

In [24]:
T='24mK'

In [25]:
S21_complex = pd.DataFrame(S21_complex, index=f)
S21_dB = np.log10(np.abs(S21_complex))*20
S21_phase = pd.DataFrame(np.unwrap(np.angle(S21_complex)), index=f)
S21_complex.to_csv(r'C:\Users\cryopxi\Documents\Alexey\T{}-complex.csv'.format(T))
S21_dB.to_csv(r'C:\Users\cryopxi\Documents\Alexey\T{}-dB.csv'.format(T))
S21_phase.to_csv(r'C:\Users\cryopxi\Documents\Alexey\T{}-phase.csv'.format(T))